# EJERCICIOS

In [3]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## EJERCICIO 0
En un documento word haz una lista de las diferentes operaciones con una breve descripción de lo que hace y un ejemplo de como se utiliza (justo la parte donde se utiliza sin contexto).

Ejemplo:

* map: aplica una funcion a todos los elementos de un rdd - rdd.map(lambda x: x * 2)

## EJERCICIO 1

Si tenemos dos RDD (A y B):
* rddA = sc.parallelize([1,2,3,4])
* rddB = sc.parallelize([3,4,5,6])

¿Cómo conseguimos los elementos que están en A y no B y los de B que no están en A? (es decir [1, 2, 5, 6]):

In [4]:
rddA = sc.parallelize([1,2,3,4])
rddB = sc.parallelize([3,4,5,6])

#  Elementos en A pero no en B
result_A_not_B = rddA.subtract(rddB)

# Elementos en B pero no en A
result_B_not_A = rddB.subtract(rddA)

# Unir ambos resultados
final_result = result_A_not_B.union(result_B_not_A)

# Mostrar el resultado
final_result.collect()


[1, 2, 5, 6]

## EJERCICIO 2

A partir de la lista implicita en el string "Perro Gato Loro Pez León Tortuga Gallina"
1.	Crea un RDD a partir de esta lista
2.	Convierte el RDD normal en un RDD de pares cuya clave sea la primera letra del animal
3.	Crea otro RDD de pares pero poniendo como clave un número incremental
4.	¿Y si queremos que el índice incremental empiece en 100?


In [10]:
animals = "Perro Gato Loro Pez León Tortuga Gallina".split()
rdd = sc.parallelize(animals)
rdd.collect()

['Perro', 'Gato', 'Loro', 'Pez', 'León', 'Tortuga', 'Gallina']

In [11]:
rdd_pairs_by_letter = rdd.map(lambda x: (x[0], x))
rdd_pairs_by_letter.collect()

[('P', 'Perro'),
 ('G', 'Gato'),
 ('L', 'Loro'),
 ('P', 'Pez'),
 ('L', 'León'),
 ('T', 'Tortuga'),
 ('G', 'Gallina')]

In [12]:
rdd_pairs_with_index = rdd.zipWithIndex()
rdd_pairs_with_index.collect()

[('Perro', 0),
 ('Gato', 1),
 ('Loro', 2),
 ('Pez', 3),
 ('León', 4),
 ('Tortuga', 5),
 ('Gallina', 6)]

In [13]:
rdd_pairs_with_index_100 = rdd.zipWithIndex().map(lambda x: (x[1] + 100, x[0]))
rdd_pairs_with_index_100.collect()

[(100, 'Perro'),
 (101, 'Gato'),
 (102, 'Loro'),
 (103, 'Pez'),
 (104, 'León'),
 (105, 'Tortuga'),
 (106, 'Gallina')]

## EJERCICIO 3

Dada la siguiente lista de compra:

lista = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]

Calcula:
1.	El total que se ha gastado por cada producto
2.	Cuánto es lo máximo que se ha pagado por cada producto

In [14]:
lista = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
rdd = sc.parallelize(lista)
total= rdd.reduceByKey(lambda x,y: x+y)
print(total.collect())

[('pan', 4), ('cereales', 3), ('agua', 2.5), ('azúcar', 1), ('filetes', 5), ('leche', 4)]


In [16]:
maximo= rdd.reduceByKey(lambda x,y: max(x,y))
print(maximo.collect())

[('pan', 3), ('cereales', 3), ('agua', 2), ('azúcar', 1), ('filetes', 5), ('leche', 2)]


## EJERCICIO 4

Ahora tenemos las cuentas de las compras de 3 días:
* día 1: pan 3€, agua 2€, azúcar 1€, leche 2€, pan 4€
* día 2: pan 1€, cereales 3€, agua 0.5€, leche 2€, filetes 5€
* día 3: filetes 2€, cereales 1€

Dada la siguiente lista de compra:

- dia1 = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',4)]
- dia2 = [('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
- dia3 = [('filetes',2), ('cereales',1)]

Responde:
1.	¿Cómo obtenemos lo que hemos gastado en cada producto?
2.	¿Y el gasto medio que hemos realizado en cada uno de ellos?

In [17]:
dia1 = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',4)]
dia2 = [('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
dia3 = [('filetes',2), ('cereales',1)]
rdd1 = sc.parallelize(dia1)
rdd2 = sc.parallelize(dia2)
rdd3 = sc.parallelize(dia3)
rdd = rdd1.union(rdd2).union(rdd3)
total= rdd.reduceByKey(lambda x,y: x+y)
print(total.collect())

[('azúcar', 1), ('filetes', 7), ('cereales', 4), ('pan', 8), ('agua', 2.5), ('leche', 4)]


In [18]:
gasto_medio= rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
print(gasto_medio.collect())

[('azúcar', 1.0), ('filetes', 3.5), ('cereales', 2.0), ('pan', 2.6666666666666665), ('agua', 1.25), ('leche', 2.0)]


## EJERCICIO 5

A partir de la lista ciudades = ['Alicante', 'Elche', 'Valencia', 'Madrid', 'Barcelona', 'Bilbao', 'Sevilla'] crea rdds con las siguientes características:

1. Sólo las ciudades que tengan la letra e en su nombre y muéstralas.
2. Ciudades que tienen la letra e y el número de veces que aparece en cada nombre.
3. Averigua las ciudades que solo tengan una única e.
4. Nos han enviado una nueva lista pero no han separado bien las ciudades. Reorganiza la lista y colocalas correctamente, y cuenta las apariciones de la letra e de cada ciudad.

    ciudades_mal = [['Alicante.Elche', 'Valencia', 'Madrid.Barcelona', 'Bilbao.Sevilla'], ['Murcia', 'San Sebastián', 'Melilla.Aspe']]

In [19]:
ciudades = ['Alicante', 'Elche', 'Valencia', 'Madrid', 'Barcelona', 'Bilbao', 'Sevilla']
rdd = sc.parallelize(ciudades)
solo_e= rdd.filter(lambda x: 'e' in x)
print(solo_e.collect())

['Alicante', 'Elche', 'Valencia', 'Barcelona', 'Sevilla']


In [34]:
veces_e= solo_e.map(lambda x: (x, x.count('e') + x.count('E'))).reduceByKey(lambda x,y: x+y)
print(veces_e.collect())


[('Sevilla', 1), ('Barcelona', 1), ('Alicante', 1), ('Elche', 2), ('Valencia', 1)]


## EJERCICIO 6

A partir de las siguientes listas:
* Inglés: hello, table, angel, cat, dog, animal, chocolate, dark, doctor, hospital, computer
* Español: hola, mesa, angel, gato, perro, animal, chocolate, oscuro, doctor, hospital, ordenador

Crea un RDD con tuplas de palabras y su traducción: 
[('hello', 'hola'),
 ('table', 'mesa'),
 ('angel', 'angel'),
 ('cat', 'gato')...]

Averigua:
1. Palabras que se escriben igual en inglés y en español
2. Palabras que en español son distintas que en inglés
3. Obtén una única lista con las palabras en ambos idiomas que son distintas entre ellas (['hello', 'hola', 'table', ...)
4. Haz dos grupos con todas las palabras, uno con las que empiezan por vocal y otro con las que empiecen por consonante.


In [64]:
Inglés=['hello', 'table', 'angel', 'cat', 'dog', 'animal', 'chocolate', 'dark', 'doctor', 'hospital', 'computer']
Español=['hola', 'mesa', 'angel','gato','perro', 'animal', 'chocolate', 'oscuro', 'doctor', 'hospital', 'ordenador']
rdd = sc.parallelize(zip(Inglés, Español))
rdd.collect()

[('hello', 'hola'),
 ('table', 'mesa'),
 ('angel', 'angel'),
 ('cat', 'gato'),
 ('dog', 'perro'),
 ('animal', 'animal'),
 ('chocolate', 'chocolate'),
 ('dark', 'oscuro'),
 ('doctor', 'doctor'),
 ('hospital', 'hospital'),
 ('computer', 'ordenador')]

In [40]:
rdd_ingles=sc.parallelize(Inglés)
rdd_español=sc.parallelize(Español)
rdd_comun= rdd_ingles.intersection(rdd_español)
rdd_comun.collect()

['angel', 'chocolate', 'doctor', 'hospital', 'animal']

In [58]:
rdd_español_distintas= rdd_español.subtract(rdd_ingles)
rdd_español_distintas.collect()

['perro', 'oscuro', 'gato', 'ordenador', 'mesa', 'hola']

In [65]:
rdd_ingles_distintas= rdd_ingles.subtract(rdd_español)
rdd_palabras_distinas= rdd_ingles_distintas.union(rdd_español_distintas)
rdd_palabras_distinas.collect()

['hello',
 'dog',
 'computer',
 'cat',
 'dark',
 'table',
 'perro',
 'oscuro',
 'gato',
 'ordenador',
 'mesa',
 'hola']

In [72]:
rdd_total= rdd_ingles.union(rdd_español)
rdd_vocales= rdd_total.filter(lambda x: x[0].lower() in 'aeiou')
rdd_vocales.collect()


['angel', 'animal', 'angel', 'animal', 'oscuro', 'ordenador']

In [73]:
rdd_total= rdd_ingles.union(rdd_español)
rdd_vocales= rdd_total.filter(lambda x: x[0].lower() not in 'aeiou')
rdd_vocales.collect()

['hello',
 'table',
 'cat',
 'dog',
 'chocolate',
 'dark',
 'doctor',
 'hospital',
 'computer',
 'hola',
 'mesa',
 'gato',
 'perro',
 'chocolate',
 'doctor',
 'hospital']

## EJERCICIO 7

A partir del fichero de El Quijote:
1. Crear un rdd con todas las palabras del documento.
2. ¿Cuantas veces aparece la palabra Dulcinea (independientemente de si está en mayúsculas o minúsculas)? ¿Y Rocinante?
3. Devuelve una lista ordenada según el número de veces que sale cada palabra de más a menos.
4. Guarda el resultado en el disco

## EJERCICIO 8

Dada una cadena que contiene una lista de nombres Juan, Jimena, Luis, Cristian, Laura, Lorena, Cristina, Jacobo, Jorge:
1. Tranforma la cadena en una lista y luego en un RDD
2. Agrúpalos según su inicial, de manera que tengamos tuplas formadas por la letra inicial y todos los nombres que comienzan por dicha letra:

    [('J', ['Juan', 'Jimena', 'Jacobo', 'Jorge']),

    ('L', ['Luis', 'Laura', 'Lorena']),
    
    ('C', ['Cristian', 'Cristina'])]
3. De la lista original, obtén una muestra de 5 elementos sin repetir valores.
4. Devuelve una muestra de datos de aproximadamente la mitad de registros que la lista original con datos que pudieran llegar a repetirse.

## EJERCICIO 9

Dada una lista de elementos desordenados y algunos repetidos, mediante rdd devolver una muestra de 5 elementos, que estén en la lista, sin repetir y ordenados descendentemente.

lista = [4,6,34,7,9,2,3,4,4,21,4,6,8,9,7,8,5,4,3,22,34,56,98]

1. Selecciona el elemento mayor de la lista resultante.
2. Muestra los dos elementos menores.

## EJERCICIO 10

En una red social sobre cine, tenemos un fichero ratings.txt compuesta por el código de la película, el código del usuario, la calificación asignada y el TIMESTAMP de la votación con el siguiente formato:

1::1193::5::978300760

1::661::3::978302109

1::914::3::978301968

Se pide:
1. Obtener para cada película, la nota media de todas sus votaciones.
2. Películas cuya nota media sea superior a 3.

## EJERCICIO 11

Tenemos las calificaciones de las asignaturas de matemáticas (notas_mates.txt), inglés (notas_ingles.txt) y física (notas_fisica.txt) de los alumnos del instituto en 3 documentos de texto. A partir de estos ficheros:
1. Crea 3 RDD de pares, uno para cada asignatura, con los alumnos y sus notas
2. Crea un solo RDD con todas las notas (cada entrada es un usuario con sus tres notas)
3. ¿Cuál es la nota más baja que ha tenido cada alumno?
4. ¿Cuál es la nota media de cada alumno?
5. ¿Cuántos estudiantes suspende cada asignatura?
         [('Matemáticas', 7), ('Física', 8), ('Inglés', 7)]
6. ¿En qué asignatura suspende más gente?
7. Total de notables o sobresalientes por alumno, es decir, cantidad de notas superiores o igual a 7.
8. ¿Qué alumno no se ha presentado a inglés?
9. ¿A cuántas asignaturas se ha presentado cada alumno?

## EJERCICIO 12

Realiza las siguientes operaciones:
1. Genera un RDD con la siguiente lista [1, 2, 3, 3, 4, 5, 6]
2. Multiplica por 2 todos los elementos del RDD anterior
3. Filtra el RDD inicial por los elementos pares
4. Muestra los elementos unicos del RDD inicial
5. Toma los elementos del RDD inicial para multiplicarlos entre si y obtener un resultado

## EJERCICIO 13

rdd = sc.parallelize([('a', 1), ('a', 2), ('a', 3), ('b', 1)])

Realiza las siguientes operaciones:
1. Obten la suma de los valores agrupados por el key
2. Ordena el RDD inicial en base al key, de forma descendente

## EJERCICIO 14

data= ['Python', 'Scala', 'Python', 'R', 'Python', 'Java', 'R' ]

rdd= sc.parallelize(data)

Cuenta cuantas veces aparece cada valor